In [1]:
import pandas as pd
from vietadminunits import parse_address, get_data
from vietadminunits.utils import to_key, to_alphanumeric
import pickle

In [2]:
df = pd.DataFrame(get_data())

In [3]:
df['long_province_alphanumeric'] = df['long_province'].apply(to_alphanumeric)
df['long_district_alphanumeric'] = df['long_district'].apply(to_alphanumeric)
df['long_ward_alphanumeric'] = df['long_ward'].fillna('').apply(to_alphanumeric)

df['province_alphanumeric'] = df['province'].apply(to_alphanumeric)
df['short_district_alphanumeric'] = df['short_district'].apply(to_alphanumeric)
df['short_ward_alphanumeric'] = df['short_ward'].fillna('').apply(to_alphanumeric)

In [4]:
for ward in ['long_ward_alphanumeric', 'short_ward_alphanumeric']:
    for district in ['long_district_alphanumeric', 'short_district_alphanumeric']:
        for province in ['long_province_alphanumeric', 'province_alphanumeric']:
            df[f"address_{ward}_{district}_{province}"] = df[ward] + df[district] + df[province]
            df[f"address_comma_{ward}_{district}_{province}"] = df[ward] + ',' + df[district] + ',' + df[province]

In [5]:
df['province_key'] = df['province'].apply(to_key, args=(1,))
df['district_key'] = df['district'].apply(to_key, args=(2,))
df['ward_key'] = df['ward'].apply(to_key, args=(3,))

In [6]:
address_cols = [col for col in df.columns if 'address' in col]

## Find wards that are the same keyword with provinces

In [7]:
wrong_provinces = []



for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)
        unit = parse_address(address, level=1)
        data = {
            'address': address,
            'province': row.province,
            'province_key': row.province_key,
            'wrong_province_key': unit.province_key,
        }
        
        if unit.province == None:
            print(address)
            wrong_provinces.append(data)
            
        elif unit.province_key != row.province_key:
            print(address)
            wrong_provinces.append(data)

In [8]:
df_wrong_provinces = pd.DataFrame(wrong_provinces)

In [9]:
df_wrong_provinces.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [10]:
df_wrong_provinces

""


In [11]:
# df_wrong_provinces[df_wrong_provinces.wrong_province_key.isna()]

In [12]:
DICT_double_check_provinces = {}
for row in df_wrong_provinces.itertuples():
    DICT_double_check_provinces[row.wrong_province_key] = df_wrong_provinces[df_wrong_provinces['wrong_province_key']==row.wrong_province_key]['province_key'].unique().tolist()

In [13]:
DICT_double_check_provinces

{}

## Find wards that are the same keyword with districts

We have to add `double_check_provinces` to module to solve province level before doing this step.

In [14]:
wrong_districts = []
for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)

        unit = parse_address(address, level=3)
    
        data = {
            'address': address,
            'province_english': row.province_english,
            'province_key': row.province_key,
            'wrong_province_key': unit.province_key,
            'district_key': row.district_key,
            'wrong_district_key': unit.district_key,
        }
        
        if unit.province == None:
            print(address)
            wrong_districts.append(data)
        elif unit.district == None:
            print(address)
            wrong_districts.append(data)
                
        elif unit.district_key != row.district_key:
            print(address)
            wrong_districts.append(data)



thitranthachhathachhatinhhatinh
xangocsonthachhatinhhatinh
xathachhaithachhatinhhatinh
xathachkenhthachhatinhhatinh
xathachsonthachhatinhhatinh
xathachlienthachhatinhhatinh
xadinhbanthachhatinhhatinh
xaviettienthachhatinhhatinh
xathachkhethachhatinhhatinh
xathachlongthachhatinhhatinh
xathachtrithachhatinhhatinh
xathachlacthachhatinhhatinh
xathachngocthachhatinhhatinh
xatuongsonthachhatinhhatinh
xathachvanthachhatinhhatinh
xaluuvinhsonthachhatinhhatinh
xathachthangthachhatinhhatinh
xathachdaithachhatinhhatinh
xathachhoithachhatinhhatinh
xatanlamhuongthachhatinhhatinh
xathachxuanthachhatinhhatinh
xanamdienthachhatinhhatinh
xatanloclochatinhhatinh
xahongloclochatinhhatinh
xathinhloclochatinhhatinh
xabinhanlochatinhhatinh
xaichhaulochatinhhatinh
xaphuluulochatinhhatinh
thitranlochalochatinhhatinh
xathachmylochatinhhatinh
xathachkimlochatinhhatinh
xathachchaulochatinhhatinh
xahodolochatinhhatinh
xamaiphulochatinhhatinh
thitranchauobinhsontinhquangngai
xabinhthuanbinhsontinhquangngai
xabinht

In [15]:
df_wrong_districts = pd.DataFrame(wrong_districts)

In [16]:
df_wrong_districts = df_wrong_districts[df_wrong_districts.province_key==df_wrong_districts.wrong_province_key]

In [17]:
df_wrong_districts

,address,province_english,province_key,wrong_province_key,district_key,wrong_district_key
0,thitranthachhathachhatinhhatinh,Ha Tinh,hatinh,hatinh,thachha,hatinh
1,xangocsonthachhatinhhatinh,Ha Tinh,hatinh,hatinh,thachha,hatinh
2,xathachhaithachhatinhhatinh,Ha Tinh,hatinh,hatinh,thachha,hatinh
3,xathachkenhthachhatinhhatinh,Ha Tinh,hatinh,hatinh,thachha,hatinh
4,xathachsonthachhatinhhatinh,Ha Tinh,hatinh,hatinh,thachha,hatinh
...,...,...,...,...,...,...
219,"datcuoc,bactanuyen,binhduong",Binh Duong,binhduong,binhduong,bactanuyen,tanuyen
220,"hieuliem,bactanuyen,binhduong",Binh Duong,binhduong,binhduong,bactanuyen,tanuyen
221,"lacan,bactanuyen,binhduong",Binh Duong,binhduong,binhduong,bactanuyen,tanuyen
222,"tanmy,bactanuyen,binhduong",Binh Duong,binhduong,binhduong,bactanuyen,tanuyen


In [18]:
df_wrong_districts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   address             224 non-null    object
 1   province_english    224 non-null    object
 2   province_key        224 non-null    object
 3   wrong_province_key  224 non-null    object
 4   district_key        224 non-null    object
 5   wrong_district_key  224 non-null    object
dtypes: object(6)
memory usage: 10.6+ KB


In [19]:
df_wrong_districts[df_wrong_districts.wrong_district_key.isna()]

,address,province_english,province_key,wrong_province_key,district_key,wrong_district_key


In [20]:
DICT_double_check_districts = {}

for province_english in df_wrong_districts['province_english'].unique():
    district_data = {}
    for wrong_district_key in df_wrong_districts[(df_wrong_districts['province_english']==province_english)]['wrong_district_key'].unique():
        tmp_district_keys = df_wrong_districts[(df_wrong_districts['province_english']==province_english) & (df_wrong_districts['wrong_district_key']==wrong_district_key)]['district_key'].unique().tolist()
        
        tmp_district_keys = sorted(tmp_district_keys, key=len, reverse=True)
        district_data[wrong_district_key] = tmp_district_keys
    
    district_data = dict(sorted(district_data.items(), key=lambda x: len(x[0]), reverse=True))
    DICT_double_check_districts[province_english] = district_data

In [21]:
DICT_double_check_districts

{'Ha Tinh': {'hatinh': ['thachha', 'locha']},
 'Quang Ngai': {'sontinh': ['binhson', 'lyson']},
 'Binh Duong': {'tanuyen': ['bactanuyen']},
 'Ho Chi Minh': {'binhthanh': ['tanbinh']}}

## Double check ward

In [22]:
wrong_wards = []
for address_col in address_cols:
    for row in df.itertuples():
        address = getattr(row, address_col)
        
        try:
        
            unit = parse_address(address, level=3)
        
            data = {
                'address': address,
                'province_english': row.province_english,
                'province_key': row.province_key,
                'wrong_province_key': unit.province_key,
                'district_english': row.district_english,
                'district_key': row.district_key,
                'wrong_district_key': unit.district_key,
                'ward_key': row.ward_key,
                'wrong_ward_key': unit.ward_key,
            }
            
            if unit.province == None:
                print(address)
                wrong_wards.append(data)
                
            elif unit.district == None:
                print(address)
                wrong_wards.append(data)
                
            elif (unit.ward == None) and (unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']):
                print(address)
                wrong_wards.append(data)
                    
            elif (unit.ward_key != row.ward_key) and (unit.district_key not in ['bachlongvi', 'conco', 'hoangsa', 'lyson', 'condao']):
                print(address)
                wrong_wards.append(data)
                
                
        except Exception as e:
            print('ERROR address:', address)
            print('ERROR info:', e)

xahaianhuyenhaihautinhnamdinh
xahaianhuyenhaihaunamdinh
xangocsonthachhatinhhatinh
xathachhaithachhatinhhatinh
xathachkenhthachhatinhhatinh
xathachsonthachhatinhhatinh
xathachlienthachhatinhhatinh
xadinhbanthachhatinhhatinh
xaviettienthachhatinhhatinh
xathachkhethachhatinhhatinh
xathachlongthachhatinhhatinh
xathachtrithachhatinhhatinh
xathachlacthachhatinhhatinh
xathachngocthachhatinhhatinh
xatuongsonthachhatinhhatinh
xathachvanthachhatinhhatinh
xaluuvinhsonthachhatinhhatinh
xathachthangthachhatinhhatinh
xathachdaithachhatinhhatinh
xathachhoithachhatinhhatinh
xatanlamhuongthachhatinhhatinh
xathachxuanthachhatinhhatinh
xanamdienthachhatinhhatinh
xatanloclochatinhhatinh
xahongloclochatinhhatinh
xathinhloclochatinhhatinh
xabinhanlochatinhhatinh
xaichhaulochatinhhatinh
xaphuluulochatinhhatinh
thitranlochalochatinhhatinh
xathachmylochatinhhatinh
xathachkimlochatinhhatinh
xathachchaulochatinhhatinh
xahodolochatinhhatinh
xamaiphulochatinhhatinh
thitranchauobinhsontinhquangngai
xabinhthuanbinh

In [23]:
df_wrong_wards = pd.DataFrame(wrong_wards)

In [24]:
df_wrong_wards = df_wrong_wards[(df_wrong_wards.province_key==df_wrong_wards.wrong_province_key) & (df_wrong_wards.district_key==df_wrong_wards.wrong_district_key)]

In [25]:
df_wrong_wards[df_wrong_wards.wrong_ward_key.isna()]

,address,province_english,province_key,wrong_province_key,district_english,district_key,wrong_district_key,ward_key,wrong_ward_key


In [26]:
df_wrong_wards

,address,province_english,province_key,wrong_province_key,district_english,district_key,wrong_district_key,ward_key,wrong_ward_key
0,xahaianhuyenhaihautinhnamdinh,Nam Dinh,namdinh,namdinh,Hai Hau,haihau,haihau,haian,haianh
1,xahaianhuyenhaihaunamdinh,Nam Dinh,namdinh,namdinh,Hai Hau,haihau,haihau,haian,haianh
102,haianhuyenhaihautinhnamdinh,Nam Dinh,namdinh,namdinh,Hai Hau,haihau,haihau,haian,haianh
103,haianhuyenhaihaunamdinh,Nam Dinh,namdinh,namdinh,Hai Hau,haihau,haihau,haian,haianh


In [27]:
DICT_double_check_wards = {}

for province_english in df_wrong_wards.province_english.unique():
    district_data = {}
    tmp_df_province = df_wrong_wards[df_wrong_wards.province_english ==province_english]
    for district_english in tmp_df_province.district_english.unique():
        tmp_df_district = tmp_df_province[tmp_df_province.district_english == district_english]
        ward_data = {}
        for wrong_ward_key in tmp_df_district.wrong_ward_key.unique():
            ward_data[wrong_ward_key] = tmp_df_district[tmp_df_district.wrong_ward_key == wrong_ward_key]['ward_key'].unique().tolist()
        
        ward_data = dict(sorted(ward_data.items(), key=lambda x: len(x[0]), reverse=True))
        district_data[district_english] = ward_data
    
    DICT_double_check_wards[province_english] = district_data

In [28]:
DICT_double_check_wards

{'Nam Dinh': {'Hai Hau': {'haianh': ['haian']}}}

In [29]:
with open('../../vietadminunits/data/parse.pkl', 'rb') as f:
    data = pickle.load(f)

data['DICT_double_check_provinces'] = DICT_double_check_provinces
data['DICT_double_check_districts'] = DICT_double_check_districts
data['DICT_double_check_wards'] = DICT_double_check_wards

with open('../../vietadminunits/data/parse.pkl', 'wb') as f:
    pickle.dump(data, f)